In [1]:
import numpy as np
from fractions import Fraction

#from numpy.linalg import inv


In [59]:
class ItemSet(object):
    def __init__(self, items):
        self.items = items
    
    def contains(self, item_set):
        return all(map(lambda x:x[0]>=-x[1], zip(self.items, item_set.items)))
    
    def __add__(self, addend): 
        return ItemSet(list(map(lambda x:x[0]+x[1], zip(self.items, addend.items))))
    
    def __len__(self):
        return sum(self.items)
    
    def __repr__(self):
        return f'({self.items})'
    
class Cast(object):
    def __init__(self, action_id, recipe, points):
        self.action_id = action_id
        self.points = points
        self.recipe = recipe

    def __repr__(self):
        return f'Cast{self.action_id}({self.recipe})'
        


In our example we will have two slots;   our initial inventory will be (0,0);
our available moves are (+2,0), (-1,+1);   and our goal is (0,1).





In [60]:
inventory = ItemSet([0,0])

goal = [0,1]

moves = [Cast(1,ItemSet([2,0]),0), Cast(2,ItemSet([-1,1]),0)]
print(f'inventory: {inventory}, goal: {goal}, moves: {moves}')


inventory: ([0, 0]), goal: [0, 1], moves: [Cast1(([2, 0])), Cast2(([-1, 1]))]


At this point I express the moves as c, d initial state is I and target state of T
we have

$$
I + A c1 + B c2 \ge T
\text{ or } 
\left(\begin{array}{ccc}c1 & c2\end{array}\right) \left(\begin{array}{ccc}A \\ B\end{array}\right) \ge T - I
$$

In our case

$$
\left(
\begin{array}{ccc}
2A -B \\
0 + B
\end{array}
\right)
\ge 
\left(\begin{array}{ccc}
0 -  0 \\ 1 - 0
\end{array}\right)
\text{or}
\left(
\begin{array}{ccc}
2 & -1 \\ 0 & 1
\end{array}
\right)
\left(\begin{array}{ccc}
A \\ B
\end{array}\right)
\ge 
\left(\begin{array}{ccc}
0 \\ 1
\end{array}\right)
$$

Clearly $B = 1$ with $A = 1$ will solve this, it must be noted the order is important and we may have some other restrictions on the casting actions.



In [123]:
M = np.array([x.recipe.items for x in moves]).T
M

array([[ 2, -1],
       [ 0,  1]])

In [101]:
x=np.linalg.solve(M, np.array([0,1]))
print(f'non-normalized: {x}')
x = x * Fraction(x.sum()).limit_denominator().denominator
print(x)

non-normalized: [0.5 1. ]
[1. 2.]


In [124]:
moves_to_make = [j for i in [[moves[ndx]]*int(x[ndx]) for ndx in range(len(moves))] for j in i]
moves_to_make

[Cast1(([2, 0])), Cast2(([-1, 1])), Cast2(([-1, 1]))]

In [125]:
inventory.contains(moves[1].recipe)

False

In [127]:
# takes moves_to_make and sorts them in order for the inventory
def sortmoves(inv:ItemSet, moves_left):
    print(f'inv: {inv}, movesleft: {moves_left}')
    for move in moves_left:
        if inv.contains(move.recipe):
            print(f'make move {move}')
            moves_left.remove(move)
            inv.items = inv.items[:]
            inv.items = np.add(inv.items, move.recipe.items)
            return [move] + sortmoves(inv, moves_left[:])
        
    return []

print(f'Inventory={inventory}, MovesToMake={moves_to_make}')
print(sortmoves(inventory, moves_to_make[:]))

Inventory=([0 2]), MovesToMake=[Cast1(([2, 0])), Cast2(([-1, 1])), Cast2(([-1, 1]))]
inv: ([0 2]), movesleft: [Cast1(([2, 0])), Cast2(([-1, 1])), Cast2(([-1, 1]))]
make move Cast1(([2, 0]))
inv: ([2 2]), movesleft: [Cast2(([-1, 1])), Cast2(([-1, 1]))]
make move Cast2(([-1, 1]))
inv: ([1 3]), movesleft: [Cast2(([-1, 1]))]
make move Cast2(([-1, 1]))
inv: ([0 4]), movesleft: []
[Cast1(([2, 0])), Cast2(([-1, 1])), Cast2(([-1, 1]))]


# Tree view

lets approach this now mapping out a tree of all options to a certain level 5 is reasonable for a 4 move set ($4^5 = 1024$)

In [58]:
class MoveNode(object):
    def __init__(self, itemset):
        self.itemset = itemset
        #self.path = path
        
    def __repr__(self):
        return f'Node{self.itemset}'

    
class MoveTree(object):
    
    def __init__(self, root):
        self.root = root
        self.branches = None # map of (ItemSet)* -> MoveNode
        
    
    
    def __repr__(self):
        branches = f', branches: {self.branches}' if self.branches else ''
        return f'Tree((root:{self.root}{branches}))'
    

def generate_branches(moves):
    # uses the set of moves to generate branches
    return  {
        move: MoveTree(MoveNode(inventory + move.recipe))
        for move in moves
    }

    
    
tree = MoveTree(MoveNode(inventory))
print(tree)

Tree((root:Node([0, 0])))


In [57]:
tree.generate_branches(moves)
print(tree)

Tree((root:Node([0, 0]), branches: {Cast1(([2, 0])): Tree((root:Node([2, 0]))), Cast2(([-1, 1])): Tree((root:Node([-1, 1])))}))


In [49]:
for m, branch in tree.branches.items():
    branch.generate_branches(moves)
    
print(tree)

Tree((root:Node([0, 0]), branches: {Cast1(([2, 0])): Tree((root:Node([2, 0]), branches: {Cast1(([2, 0])): Tree((root:Node([2, 0]))), Cast2(([-1, 1])): Tree((root:Node([-1, 1])))})), Cast2(([-1, 1])): Tree((root:Node([-1, 1]), branches: {Cast1(([2, 0])): Tree((root:Node([2, 0]))), Cast2(([-1, 1])): Tree((root:Node([-1, 1])))}))}))


In [55]:
tree.branches

{Cast1(([2, 0])): Tree((root:Node([2, 0]), branches: {Cast1(([2, 0])): Tree((root:Node([2, 0]))), Cast2(([-1, 1])): Tree((root:Node([-1, 1])))})),
 Cast2(([-1, 1])): Tree((root:Node([-1, 1]), branches: {Cast1(([2, 0])): Tree((root:Node([2, 0]))), Cast2(([-1, 1])): Tree((root:Node([-1, 1])))}))}